# Summary by city of proximity analysis data (res9)

This notebook sumarizes in a csv pobtot and average max_times to each eje and amenity by city.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download available cities

In [3]:
mun_schema = 'metropolis'
mun_table = 'metro_gdf_2015'

query = f"SELECT * FROM {mun_schema}.{mun_table}"
mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')

city_list = mun_gdf.city.unique()

# Show
city_list

array(['Aguascalientes', 'Ensenada', 'Mexicali', 'Tijuana', 'La Paz',
       'Campeche', 'Laguna', 'Monclova', 'Piedras Negras', 'Saltillo',
       'Colima', 'Tecoman', 'Tapachula', 'Tuxtla', 'Chihuahua',
       'Delicias', 'Juarez', 'Parral', 'ZMVM', 'Durango', 'Celaya',
       'Guanajuato', 'Leon', 'Moroleon', 'San Francisco', 'Acapulco',
       'Chilpancingo', 'Pachuca', 'Tula', 'Tulancingo', 'Guadalajara',
       'Ocotlan', 'Vallarta', 'Tianguistenco', 'Toluca', 'Piedad',
       'Morelia', 'Zamora', 'Cuautla', 'Cuernavaca', 'Tepic', 'Monterrey',
       'Oaxaca', 'Tehuantepec', 'Puebla', 'Tehuacan', 'Teziutlan',
       'Queretaro', 'Cancun', 'Chetumal', 'Rio Verde', 'SLP', 'Culiacan',
       'Mazatlan', 'Guaymas', 'Hermosillo', 'Nogales', 'Villahermosa',
       'Victoria', 'Matamoros', 'Nuevo Laredo', 'Reynosa', 'Tampico',
       'Tlaxcala', 'Acayucan', 'Coatzacoalcos', 'Cordoba', 'Minatitlan',
       'Orizaba', 'Poza Rica', 'Veracruz', 'Xalapa', 'Merida',
       'Zacatecas'], dtype

## Create summary data

In [5]:
summary = pd.DataFrame()
i = 0

for city in city_list:
    
    print(f"Starting processing {city}, {i}/74")
    
    # Download proximity analysis data
    prox_schema = 'prox_analysis'
    prox_table = 'proximityanalysis_20_ageb_hex8'
    query = f"SELECT * FROM {prox_schema}.{prox_table} WHERE \"city\" LIKE \'{city}\'"
    prox_gdf = aup.gdf_from_query(query, geometry_col='geometry')

    print(f"Downloaded proximity data for {city}")
    
    # Add city data
    summary.loc[i,'city'] = city
    
    # Calculate time_pond
    pob_0a5_tot = prox_gdf.pob_0a5.sum()
    prox_gdf['time_pond'] = (prox_gdf['pob_0a5']/pob_0a5_tot)*(prox_gdf['max_idx_15_min'])
    time_pond = prox_gdf['time_pond'].sum()
    
    # Add time_pond data
    summary.loc[i,'time_pond'] = time_pond
    
    print(summary)
        
    i = i+1

Starting processing Aguascalientes, 0/74
Downloaded proximity data for Aguascalientes
             city      score
0  Aguascalientes  75.355918
Starting processing Ensenada, 1/74
Downloaded proximity data for Ensenada
             city       score
0  Aguascalientes   75.355918
1        Ensenada  160.824659
Starting processing Mexicali, 2/74
Downloaded proximity data for Mexicali
             city       score
0  Aguascalientes   75.355918
1        Ensenada  160.824659
2        Mexicali  163.189181
Starting processing Tijuana, 3/74
Downloaded proximity data for Tijuana
             city       score
0  Aguascalientes   75.355918
1        Ensenada  160.824659
2        Mexicali  163.189181
3         Tijuana   85.015382
Starting processing La Paz, 4/74
Downloaded proximity data for La Paz
             city       score
0  Aguascalientes   75.355918
1        Ensenada  160.824659
2        Mexicali  163.189181
3         Tijuana   85.015382
4          La Paz  124.330408
Starting processing Campec

## Show and save data

In [7]:
summary

,city,time_pond
0,Aguascalientes,75.355918
1,Ensenada,160.824659
2,Mexicali,163.189181
3,Tijuana,85.015382
4,La Paz,124.330408
...,...,...
69,Poza Rica,82.223471
70,Veracruz,84.703845
71,Xalapa,60.714642
72,Merida,77.246742


In [8]:
summary.to_csv("../../data/external/temporal_fromjupyter/pob0a5_prox_promedioponderado.csv")